In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.datasets as datasets
#from vae import VAE
from vaecnn import VAECNN
#from util import train,test

In [2]:
from matplotlib.pyplot import imshow
import numpy as np
import PIL
from PIL import Image

In [3]:
transform = transforms.Compose([
     transforms.ToPILImage(),
     transforms.RandomCrop(80),
     transforms.Resize((96,96)),
     transforms.RandomHorizontalFlip(),
     transforms.ColorJitter(hue=.05, saturation=.05),
     transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
     transforms.ToTensor()
])

In [4]:
def tripletLoss(anchor, positive, negative, size_average=True, margin = 0.5):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + margin)
        return losses.mean() if size_average else losses.sum()

In [5]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar,z):
    #BCE = F.binary_cross_entropy(recon_x, x, reduction='sum') #.view(-1, 784)
    BCE = F.mse_loss(recon_x, x, size_average=False)/100
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = (-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())) / 100
    
    anchor = z[:z.shape[0]/2]
    positive = z[z.shape[0]/2:]
    negative = torch.cat((anchor[-1:], anchor[:-1]))
#     print(anchor.shape)
#     print(positive.shape)
#     print(negative.shape)
    TL = tripletLoss(anchor, positive, negative, size_average=True) 
    
    print('BCE',BCE)
    print('KLD',KLD)
    print('TL',TL)
    
    return  BCE + 0.1 * (KLD) + 0.1 * TL

In [6]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data_T = torch.stack([transform(d) for d in data])
        data = torch.cat( (data,data_T), 0 )
        data = data.to(device)
        #print(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar,z = model(data)
        loss = loss_function(recon_batch, data, mu, logvar,z)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar, z = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar,z).item()
#             if i == 0:
#                 n = min(data.size(0), 8)
#                 comparison = torch.cat([data[:n],
#                                       recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
#                 save_image(comparison.cpu(),
#                          'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [7]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [8]:
args = Namespace(
    batch_size = 100,
    epochs = 10,
    cuda = True,
    seed = 2019,
    log_interval = 1)

In [9]:
torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")

In [10]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

trainDataDir = '/scratch/um367/DL/data/sample_4/train'
valDataDir = '/scratch/um367/DL/data/sample_4/val'
# trainDataDir = '/scratch/um367/DL/data/sampledata/supervised/train'
# valDataDir = '/scratch/um367/DL/data/sampledata/supervised/val'


train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(trainDataDir, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valDataDir, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

In [11]:
print(len(train_loader.dataset))

128000


In [12]:
model = VAECNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
## load parameters
# to load
checkpoint = torch.load('vaeSiamTest.pth.tar')
model.load_state_dict(checkpoint['model_state_dict'])

In [14]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    #test(epoch)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'vaeSiamTest.pth.tar')
#     with torch.no_grad():
#         sample = torch.randn(64, 20).to(device)
#         sample = model.decode(sample).cpu()
#         save_image(sample.view(64, 1, 28, 28),
#         'results/sample_' + str(epoch) + '.png')

/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BCE tensor(4698.8848, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(1.4574, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(3.1821, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [0/128000 (0%)]	Loss: 23.496743
BCE tensor(4413.4399, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(2.0585, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(1.8376, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [200/128000 (0%)]	Loss: 22.069148
BCE tensor(4437.3740, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(2.5861, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(3.9696, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [400/128000 (0%)]	Loss: 22.190149
BCE tensor(4643.2837, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(3.4202, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(2.3732, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [600/128000 (0%)]	Loss: 23.219314
BCE tensor(4531.4185, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(

BCE tensor(3474.6599, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(78.7083, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(10.8242, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [7000/128000 (3%)]	Loss: 17.418066
BCE tensor(3569.1577, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(95.9449, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(22.7001, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [7200/128000 (3%)]	Loss: 17.905111
BCE tensor(3477.8154, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(135.0980, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(21.8954, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [7400/128000 (3%)]	Loss: 17.467573
BCE tensor(3507.0430, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(161.9765, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(26.9095, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [7600/128000 (3%)]	Loss: 17.629658
BCE tensor(3240.7510, device='cuda:0', grad_fn=<DivBackwar

Train Epoch: 1 [13800/128000 (5%)]	Loss: 11.234431
BCE tensor(2254.5222, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(366.8944, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(2.5394, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [14000/128000 (5%)]	Loss: 11.457328
BCE tensor(2119.2332, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(389.4360, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(3.0220, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [14200/128000 (6%)]	Loss: 10.792395
BCE tensor(2265.6113, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(375.9180, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(1.5981, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [14400/128000 (6%)]	Loss: 11.516815
BCE tensor(2052.0518, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(370.7285, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(3.1918, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [14600/128000 (6%)]	Loss: 10.447218
BCE t

Train Epoch: 1 [20800/128000 (8%)]	Loss: 8.595234
BCE tensor(1754.6752, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(386.4341, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(5.7234, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [21000/128000 (8%)]	Loss: 8.969455
BCE tensor(1830.0184, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(385.7382, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(5.9260, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [21200/128000 (8%)]	Loss: 9.345925
BCE tensor(1852.2632, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(389.4325, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(3.4456, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [21400/128000 (8%)]	Loss: 9.457755
BCE tensor(1791.8444, device='cuda:0', grad_fn=<DivBackward0>)
KLD tensor(412.1341, device='cuda:0', grad_fn=<DivBackward0>)
TL tensor(2.6967, device='cuda:0', grad_fn=<MeanBackward1>)
Train Epoch: 1 [21600/128000 (8%)]	Loss: 9.166637
BCE tensor

...

IndexError: string index out of range

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/share/apps/python/2.7.12/intel/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/share/apps/python/2.7.12/intel/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/share/apps/python/2.7.12/intel/lib/python2.7/inspect.py", line 454, in getsourcefile
    if h

IndexError: string index out of range

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 416, in execute_request
    ident=ident)
  File "/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/jupyter_client/session.py", line 748, in send
    stream.send_mul